In [1]:
import random
import functools
import timeit
import time
import pyrankvote
from array import *
import gurobipy as gp
from gurobipy import GRB
import itertools
from stvpoll.scottish_stv import ScottishSTV
import math
import pandas as pd
import math
import copy
import numpy
from decimal import *
from decimal import Decimal
from pyrankvote import Candidate, Ballot
import inspect
from stvpoll.__init__ import ElectionRound

In [2]:

#number of votes
#length of each ballot
most=2
x=7
#number of the candidates
#m = rndomBallotGen(n,l,x)

In [3]:
def tolist(listuple):
    ans=[]
    for i in ((listuple)):
        ans.append(list(i))
    return ans

In [4]:
    lstt = list(range(x))
    permt=list(itertools.permutations(lstt))
    permt.sort()
    permt=copy.deepcopy(tolist(permt))

In [5]:
def zetta(order,k,r):
    cand=[]
    for i in range(r):
        cand.append(order[i])
#    print(cand)
    
    lst = list(order)
    perm=list(itertools.permutations(lst))
    perm.sort()
    ans=[]
    ek=k
    for i in range(len(perm)):
        ind=perm[i].index(ek)
#        print(ind)
        ok=True
    #    print(i,perm[i],ind)
        for j in range(0,ind):
            if((perm[i][j] not in cand)):
            #    print(perm[j])
                ok=False
                break
        if(ok):
            ans.append(i)
    
    
        
    return ans

In [6]:
print(zetta([0,1,2],2,2))

[0, 1, 2, 3, 4, 5]


In [7]:
def convert(a,b):
    ans=list(copy.deepcopy(a))
 #   print("a=",a)
    for i in a:
        if i not in b:
            ans.remove(i)
    return ans

In [8]:
def rndomBallotGen(most):
    votes  = []
    for i in range(math.factorial(x)):
        votes.append(random.randint(0,most))
        
    return votes

In [9]:
def ILP(votes,order):
    #define model
    largeValue = 1000
    noc=len(order)
    model = gp.Model("margin")
    model.setParam("OutputFlag",0)

    ted=math.factorial(noc)
    if(len(votes)!=math.factorial(len(order))):
        vt=[0]*ted
        ls2 = copy.deepcopy(order)
        pr2=tolist(itertools.permutations(ls2))
        pr2.sort()
        for i in range(len(permt)):
            rw=convert(permt[i],order)
#            print(order,rw)
#            print(votes,len(perm),len(votes))
            vt[pr2.index(rw)]+=votes[i]
        votes=copy.deepcopy(vt)
        lst=copy.deepcopy(ls2)
            
        
#    model.params.NonConvex = 2
    b = model.addVar(vtype=GRB.INTEGER,name='b')
    # x[i,j] = a binary variable that is 1 iff candidate j is eliminated before candidate i
    #ai,j a variable representing the number of ballots < i, j > that are added
    p = model.addVars(ted,vtype=GRB.INTEGER,name='p')
    m = model.addVars(ted,vtype=GRB.INTEGER,name='m')
    y = model.addVars(ted,vtype=GRB.INTEGER,name='y')
    
    for i in range(ted):
        model.addConstr(y[i]<=sum(votes))
        model.addConstr(votes[i]+p[i]-m[i]==y[i])
        model.addConstr(0<=y[i])
        model.addConstr(m[i]<=votes[i])
        model.addConstr(0<=m[i])
        model.addConstr(0<=p[i])
    model.addConstr(sum(p[i] for i in range(ted)) == sum(m[i] for i in range(ted)))
    for i in range(noc):
        for j in range(i+1,noc):
            set1=zetta(order,order[i],i)
            set2=zetta(order,order[j],i)
            p1=gp.quicksum(y[l] for l in set1)
            p2=gp.quicksum(y[l] for l in set2)
       #     print(set1,set2)
            model.addConstr(p1<=p2)
    model.addConstr(b == gp.quicksum(p[i] for i in range(ted)))
    model.setObjective(b, GRB.MINIMIZE)
    model.optimize()
    
    pa=list(p[i].X for i in range(ted))
    ya=list(y[i].X for i in range(ted))
    ma=list(m[i].X for i in range(ted))
  #  print(pa,ya,ma)
  
    #print(myvars)

    return model.getObjective().getValue()

In [10]:
# lst = list(range(x))
# perm=list(itertools.permutations(lst))
# perm.sort()
# print(perm)

In [11]:
def margin(votes,w):
    ans=1000*1000
    order=[]
    lst = list(range(x-1))
    perm=list(itertools.permutations(lst))
    perm.sort()
    for i in perm:
        i=list(i)
        for j in range(x-1):
            if(i[j]>=w):
                i[j]+=1
        i.append(w)
        jav=ILP(votes,i)
        if(ans>jav):
            ans=jav
            order=i
        
    return ans,order

In [12]:
votes=rndomBallotGen(most)
#print(votes)
#print(margin(votes,0))

In [13]:
def optmar(votes,w):

    ans=1000*1000
    order=[]
    pr=[]

#    print("perm=",perm)

#    print(perm[0])
    pr.append([0,[w]])
    while(len(pr)):
        it=copy.deepcopy(pr[0])
        if(len(it[1])==x):
#            print(pr)
            return it[0],it[1]
        for i in range(x):
            if(i not in it[1]):
                it2=[i,*it[1]]
                secprm=list(itertools.permutations(it2))
                secprm.sort()
                secprm=copy.deepcopy(tolist(secprm))
                vt=[0]*math.factorial(len(it2))
            #    print("it2 =",it2)
                for j in range(len(permt)):
                    aftcnv=copy.deepcopy(convert((permt[j]),it2))
           #         print(aftcnv)
           #         print(secprm)
           #         print(perm[j],it2)
                    vt[secprm.index(aftcnv)]+=votes[j]
                pr.append((ILP(vt,it2),it2))
        pr.remove(pr[0])
        pr.sort()
                    
        
        

In [14]:
# start=time.time()
# print(optmar(votes,0))
# end=time.time()
# print(end-start)

In [15]:
# start=time.time()
# print(margin(votes,0))
# end=time.time()
# print(end-start)

In [16]:
def deltasce(votes,order,c,e):
    S=copy.deepcopy(order)
    S.append(e)
    ans=0
#     lst=list(i for i in range(x))
#     perm=list(itertools.permutations(lst))
#     perm.sort()
# #    print("perm=",perm)
#     perm=copy.deepcopy(tolist(perm))
    for i in range(len(permt)):
        jayg=copy.deepcopy(convert(permt[i],S))
        if(jayg[0]==c):
            ans+=votes[i]
    return ans
    

In [17]:
def f(e,votes):

    ans=0
    for i in range(len(permt)):
        if(permt[i][0]==e):
            ans+=votes[i]
    return ans
    
    

In [18]:
def l2(c,e,order,votes):
    return math.ceil((f(e,votes)-deltasce(votes,order,c,e))/2)

In [19]:
def lb2(order,votes):
    ans=0
    for c in order:
        for e in range(x):
            if(e not in order):
                ans=max(ans,l2(c,e,order,votes))
    return ans
                

In [20]:
def expand(l,order,U,Q,votes):
    if(len(order)==x):
#        print(votes,z,order)
        return l,Q,order
    for c in range(x):
        if c not in order:
            orpr=[c,*order]
            lpr=max(l,lb2(orpr,votes))
            if lpr<U:
                m=ILP(votes,orpr)
                lpr=max(lpr,m)
            if lpr<U:
 #               print("lpr=",lpr,"orpr=",orpr,"ILP=",ILP(votes,orpr))
                Q.append([lpr,orpr])
    return U,Q,order

In [21]:
def blom(votes,w):
    ans=1000*1000
    pr=[]
    order=[]
#    print(perm[0])
    pr.append([0,[w]])
    while(len(pr)):
        
        pr.sort()
#        print(pr)
        it=copy.deepcopy(pr[0])
    #    print(it[0],it[1])
        pr.remove(pr[0])
        
        if(it[0]<ans):
            ans,pr,order=expand(it[0],it[1],ans,pr,votes)
    return ans,order


In [22]:
# start=time.time()
# print(blom(votes,0))
# end=time.time()
# print(end-start)

In [ ]:
ted=0
maxi=10
t1=0
t2=0
t3=0
while(ted<maxi):
    ted+=1
    votes=rndomBallotGen(most)
    start=time.time()
    o1,o2=optmar(votes,0)
    end=time.time()
    t1+=end-start
    start=time.time()
    b1,b2=blom(votes,0)
    end=time.time()
    t2+=end-start
    # start=time.time()
    # d1,d2=margin(votes,0)
    # end=time.time()
    # t3+=end-start
    if((o1!=b1)):
        print(votes)
        print(d1,d2)
        print(o1,o2)
        print(b1,b2)
        print("\n")
    print(t1,t2)
    

Set parameter Username
Academic license - for non-commercial use only - expires 2023-05-26
